# Intro

<b>user_guide</b>, introduces 'light_guide' step by step, and teaches you how to run simulations- as well as test (relativily) simple changes to the system.

* I encourage you to be patient, and follow the instructions in this notebook in a chronological order- even if you are only interested in one specific feature.

* Surely you would rather invest a couple hours reading this notebook, than trying to figure out unexpected errors in the simulation.

* <i>Most importantly- never change anything in the code, that you don't know how to change back.</i>






# Getting familiar

* You would first want to get a feel of how the program works, and how the computer model of the lightguide corresponds to the real life lightguide.

* For that you should start by running the function <b>sanity_check_sources(rays)</b>. You input a number of rays to generate, and the function shows you statistics about those rays (spatial and angular distribution). This should mimic the behavior of photons, coming out of quartz fibers in the ZDC.

* Next up, you would want to convince yourself that ray tracing works in my program. We can start with a visual demonstration using <b>sanity_check_LG(surface,parallel=False,cones=False)</b>. This function takes any surface of the form $z=f {x\choose y}$ (you only input $f {x\choose y}$) and plots a trace of a random photon (from the aformentioned distribution) that is allowed to reflect at most one time off of the lightguide surface.
 * The program uses [sympy.Vector](https://docs.sympy.org/latest/modules/vector/index.html) for computations, such that the <u>input surface will have to be written in sympy notation</u>. Here we denote the coordinate variables: z--->N[2], y--->N[1], x--->N[0]. For future reference, the respective <u>unit vectors</u> are N.z , N.y , N.x.
 * A basic example is given with the variable <b>winston_cone</b> which represents a paraboloid: $N[2] = 7.38 -0.65(N[0]^2 +N[1]^2) $. Look at the equation by running 'display(winston_cone)'. The focal point of this paraboloid is at the center of the PMT, which makes it the default surface for the program.
 * By changing 'parallel=True' you can trace light rays that come in at parallel to the z-axis. Together with <b>winston_cone</b>, each and every ray you trace should hit the PMT. 'great success' means the ray hit the PMT without first being reflected, 'very nice success' means detection after reflection.
 * Using 'parallel=False', you should that only rarely light rays are detected. Now you can switch 'cones=True' and see some improvement (how much improvement depends largely on <b>cone_grid</b>- see the next cell)
 
* Hopefully you followed all steps thus far, and are wondering what is the purpose of 'cones=True' on the last step. This allows photons to first reflect off of a <b>dog cone</b> (if the randomly picked quartz rod, has one), before the visual trace. A lot of the time, the dog cones "correct" the trajectory of the photons such that they can be concentrated into the PMT.

* You can see a visual demonstration using <b>sanity_check_cones()</b>.
 * This one treats all rods as having cones, just for the sake of demonstration. 

# Detector Construction
* The second cell in <b>light_guide</b> determines the geometry of the system. The default values, are a combination of the current segmentation- and proposed changes to accomodate the dog cones. The program is written in units of [cm]. <u>This cell allows you to make small changes and test their effect</u>

* It's preferable to comment out (#) default values (rather than deleting), before adding your own changes. That way you can always go back if you need to. Also, it will help you to stay "in sync" with the program- you should pay attention to the manner in which different variables, originally, are calculated and/or represented.

* dog cone properties:
 * 'rf'- the radius of the fibers, it effect the cone length and height, and also the spatial distribution of the light (with respect to the fibers).
 * 'rc'/'RC' - the small/large radii of the dog cones.
 * 'a' - the angle of the dog cone. Together with the radii, those parameters completely define the dog cones.
 * 'hc'/'lc'- the height and length of the cones are determined using the previous variables.
 * 'cone_grid'- this is a numpy matrix. You need to make sure it's shape is compatible with the number of fibers! Ergo, the shape should be (n_of_chambers,n_fibers_per_chamber). If you want fiber (m,k) to have a dog cone, cone_grid[m,k] should be set to True or 1. Otherwise, False or 0.
 
* general properties of the system:
 * 'xp','yp'- determine the size of a rectangular PMT.
 * 'h'- determines the location of the PMT- is sits on plane z=h.
 * 'xb','yb'- define a rectangular boundary for the rod grid. These are almost expendable.
 * 'p_absorb'- probability of photon absorption per reflection off of the light-guide surface (for dog cones we neglect this probability).
 
* properties of the light sources
 * Here we create a defined grid for the quartz rods. 
 * However you choose, you need to define a numpy array for x positions of the chambers.
 * Similarily, you need to define a numpy array for y positions of the rods, along each chamber.
 * The final grid 'source_grid' should be np.meshgrid(x_vec,y_vec). 
 * In addition, a numpy matrix 'offset_grid' defines the z position of the sources.
 * In this way, the program knows <u>where the 'm,k' fiber is located</u>: x=source_grid[0][k,m],y=source_grid[1][k,m], z=offset_grid[m,k]. 

# Testing Existing Reflectors

* You can use <b>Test_Efficiency(eq,N_iter,N_ray,N_proc,N_bounce,parallel=False,cones=True)</b> to test the efficiency of the light guide under specified conditions. By efficiency, I mean the fraction of generated photons, to reach the PMT.
    * 'eq'- you need to specify the surface of the light guide. <u>Unlike sanity_check_LG</u>, we now want the surface to be given as a level set of a 3D function. e.g: $\mathbf{f\begin{pmatrix}x \\ y \\z \end{pmatrix}=0}$. For example you could use the default variable 'winston_cone' with a small tweak; To make it into an expression of the form $f\begin{pmatrix}x \\ y \\z \end{pmatrix}=0$, you input <b>'eq=N[2]-winston_cone'</b>.
    * 'N_iter' and 'N_ray'- to generate insightful statistics, the function doesn't count all the rays at once. Instead, it breaks up the testing process into 'N_iter' batches of 'N_ray' photons. I have found 10,000 rays (total) to give fairly consistent results, in less than 5 minutes- and for that reason N_iter=N_ray=100 is the default choice.
    * 'N_proc'- the function uses python's '[multiprocessing](https://docs.python.org/3/library/multiprocessing.html)' to run calculations in parallel and save time. Depending on the number of cores your CPU has, you should choose N_proc to try and run more/less processes in parallel. Some trial and error initially could help you optimize the run time for your machine (ctrl+shift+esc will show you CPU usage, aim to maximize it). For my laptop with intel I5 CPU & 4 cores, N_procs=8-10 gave the best results.
    * 'N_bounce'- in this function only, you could trace the rays for more than 1 reflection off of the surface. Keep in mind, on each reflection a photon may be absorbed instead of reflecting (depending on the probability you set up). When testing this on winston_cone surface, I have found very little impact for N_bounce > 1, on the final efficiency. In fact, a large fraction of rays ended up flipping direction and start traveling back toward the rods after 3,4 bounces.
    * 'parallel' and 'cones'- same purpose as last time.

* Prove to yourself that this one works! Test the winston cone with parallel=True, N_bounce=1. Don't expect to get 100%! After all, we have some photons getting absorbed in the reflection process. Change p_absorb=0 and try again. Don't forget to change it back when you're done.

* Aside from printing the histogram and the efficiency, the function returns an array. Each element in the array, counts the # of photons to get detected in it's respective batch. You can save and analyze the data yourself.

* If you are curious about the dog cones, try the winston cone with parallel=False- once with cones=False, once with cones=True. It really works.


# Optimization

### (This is the trickiest part of the guide, pay attention and go slow!)

* There are two functions that you need to know, that go hand in hand; <b>Optimize(eq,N_iter,N_ray,N_proc,parallel=False,cones=False)</b>, and <b>Optimization_Results(ef_list,surface_list,coef_mat)</b>.

* As it happens, the output of <b>Optimize()</b> is exactly the input of <b>Optimization_Results()</b>. So most often, you will probably want to save the output of the first function and immidiately pass it into the second function.

* <b>Optimize()</b> takes a surface level set $f\begin{pmatrix}x \\ y \\z \end{pmatrix}=0$, shoots a ray at it, and determines the required surface at the contact point in order to ensure detection. After many rays, the function uses all of those calculations to fit a new surface- and does it over and over again to try an converge on an ideal surface. Now, how do we use it?

* The input of <b>Optimize()</b> is the same as <b>Test_Efficiency()</b> from the last cell, with two notable exceptions:
    * 'N_iter'- it's not very helpful to do many iterations. Usually, when convergence happens- it happens within 10-20 iterations.
    * 'N-rays'- if parallel=False (usually it should be), you would ideally want as many rays as possible in each iteration- to get sufficient data for the fitting. Of course, you don't want too many- because then it would take a lot of time for the function to run. I have found that somewhere between $10^4-10^5$ rays per iteration works well, maybe start with $10^4$ and if the function doesn't converge well- gradually work your way up.
    
* Try it first with 'N_iter=10,N_ray=1000,parallel=True'. Don't go changing anything yet- start with an initial condition that is a paraboloid: $f\begin{pmatrix}x \\ y \\z \end{pmatrix}=N[2]+a_0\cdot(N[0]^2 + N[1]^2)-a_1=0$ or a cone: $f\begin{pmatrix}x \\ y \\z \end{pmatrix}=N[2]+a_0\cdot(N[0]^2 + N[1]^2)^{0.5}-a_1=0$. Try to keep it reasonable (doesn't block the PMT or the sources, facing the right way). Maybe check first by using 'sanity_check_LG'. For reasonable initial conditions, I have found the function converges to 'winston_cone' pretty often. See for yourself. The function prints out its progress.

* Now comes the time, where you have made your own changes to the Detector Construction, and would maybe want to optimize the surface under your own conditions. You'll find sometimes you want to change the functional form of $f\begin{pmatrix}x \\ y \\z \end{pmatrix}=0$. The rules are:
    * You have to choose a radially symmetric expression. That means- no occurences of N[0], or N[1] seperately. Always keep them together using $\rho = (N[0]^2 + N[1]^2)^{0.5}$ (explicitly, not via the letter $\rho$)
    * Try and make sure the initial condition makes sense. The function always constrains the surface to pass through (xp,yp,h)- maybe require your initial condition to pass there too.
    * In an expression of the form $f\begin{pmatrix}\rho \\z \end{pmatrix}=0$, where each term has a coefficient ($a_0\cdot z^2 + a_1\cdot \rho^2 +... =0$) you can always set one of the coefficients to be equal 1. That will make your life easier, and also the regression easier.
    * Now that the ground rules have been covered- you can comment out the function <b>gen_func()</b>, and make a function with the same name. This returns the functional form you have chosen for $f\begin{pmatrix}\rho \\z \end{pmatrix}=0$, and takes as input a certain number of parameters. Don't bother with a constant of integration.
    * This is the tricky part, so watch closely. You should also comment out <b>imp_func()</b>, I suggest copying it word for word to your function. You should only change the names/number of parameters that the function takes in; and the second line of the function. The second line should be, parametrically: vec=$-\frac{\partial_{\rho}f}{\partial_zf}$. In the numerator, partial derivative of $f\begin{pmatrix}\rho \\z \end{pmatrix}$ with respect to $\rho$. The denominator- partial derivative with respect to z (N[2]).By the implicit function theorem, this is the implicit derivative $\frac{dz}{d\rho}$. Convince yourself that the original 'vec=...' that you commented out, matches <b>gen_func()</b> by this rule.
    * Now the easy part- in <b>Optimize</b>, search for lines which are commented with 3 '###'. These should be changed such that the number, order, and names of the parameters are compatible with <b>gen_func()</b> you have set up. For example, the first line has 'N_coef=..'. If you have 'P' parameters (an integer), you set one coefficient in <b>gen_func()</b> to 1- and the constant of integration is set automatically: then you should set 'N_coef= P +2'. 
    * If you carefully changed everything, you should be good to go. Try to run it, and if an error comes out- check that you followed all the steps above.
    * You can also change <b>Optimization_Results()</b>, I would imagine some people would like to make the plots a bit prettier.

# Errors in Optimization

* The first thing to note, isn't an error. What should we do if the function hadn't converged?
    * First- try again with 'N_ray' bigger.
    * Different initial conditions.
    * Change 'init_guess' and/or 'bounds' in the line 'v,covm = curve_fit(...)'
    * The constant of integration 'const' is determined by constraining the function to pass through the vertices of the PMT. Maybe a different constraint could work for you, try it!
    * Different functional form.
* Sometimes the function converges on some surface that doesn't make sense.
    * Print out 'surface_list' from the output. 
    * If the function is just the same one from the beginning, maybe you accidentaly changed the line that updates the surface.
    * If the coefficients are all (except for constant of integration) exactly on the edges of 'bounds=(..)', try first to change the bounds.
    * If that doesn't help, it means that 'curve_fit' can't find a good set of parameters. Sometimes it's because the data we gave it doesn't make sense- probably a mistake somewhere, start by checking 'gen_func()' and 'imp_func()' that you set up. Check that the surface you entered as 'eq' follows all our rules.
* "index -1 is out of bounds for axis 0 with size 0"
    * This happens when the data from our rays, came back empty. e.g no simulated rays hit the surface in a reasonable location.
    * Hypothetically, that shouldn't happen for 'normal' surfaces but first try increasing 'N_ray', maybe eventually something would hit.
    * If you still get the error, it may be that 'curve_fit' has run off and given us a surface which is not appropriate for the system- too large, maybe facing in the wrong direction. Try to make 'bounds()' smaller, change 'init_guess'.
    * Maybe there is a mistake somewhere. Mistakes in 'gen_func,imp_func' can make this happen.
    * Try different initial surface or different functional form.
* "Residuals not finite for initial values"
    * This is once again an error that has to do with 'curve_fit'.
    * Try the same things like the previous error- more rays, different bound/init_guess, check for mistakes.
    